In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
img_paths = glob('./images/*.jpg')

In [3]:
img_paths

['./images/1.jpg']

In [5]:
all_invoices = pd.DataFrame(columns=['id','text'])

for img_path in  tqdm(img_paths,desc='invoice'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(img_path)
    # extract data and text 
    image = cv2.imread(img_path)
    data = pytesseract.image_to_data(image)
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(int)

    useFulData = df.query('conf >= 30')

    # Dataframe
    invoice = pd.DataFrame()
    invoice['text'] = useFulData['text']
    invoice['id'] = filename
    
    # concatenation
    all_invoices = pd.concat((all_invoices,invoice))

invoice: 100%|████████████████████████████████████| 1/1 [00:01<00:00,  1.48s/it]


In [6]:
all_invoices

,id,text
5,1.jpg,INVOICE
9,1.jpg,#
10,1.jpg,10001
12,1.jpg,Date:
13,1.jpg,May
14,1.jpg,"8,"
15,1.jpg,2022
18,1.jpg,Bill
19,1.jpg,To:
22,1.jpg,Saifullah


In [7]:
all_invoices.to_csv('invoice.csv', index=False)